In [4]:
import os
import json
import logging
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.tools import FunctionTool
import streamlit as st
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

with open(os.path.join(os.getcwd(), "config.json"), 'r') as f:
    config = json.loads(f.read())

llm = GoogleGenAI(
    model=config["llm"]["model"],
    temperature=config["llm"]["temperature"],
    api_key=os.getenv("GOOGLE_API_KEY"),
)

In [5]:
# Import Libraries for Analysis
import os
import pandas as pd
import numpy as np
import openpyxl as pyx

# Import Libararies for Visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Use Windows Installed fonts for rendering devnaagri text
plt.rcParams['font.family'] = ['Nirmala UI']
plt.rcParams['axes.unicode_minus'] = False

In [6]:
def query_dataframe(code: str):
    """Executes Python code on the dataframe `df` and returns result + code."""
    try:
        # Eval the result
        print("Generated Code: ", code)
        result = eval(code)
        return result
    except Exception as e:
        return f"Error: {e}"

In [7]:
# def visualize_data(code: str):
#     """
#     Executes code for seaborn/matplotlib plot using global df,
#     saves the figure to disk, and returns image path.
#     """
#     global df
#     image_filename = "seaborn_plot.png"
#     image_path = os.path.join("plots", image_filename)

#     try:
#         print("Generated Code:\n", code)
#         exec(code)
#         fig = plt.gcf()
#         os.makedirs("plots", exist_ok=True)
#         fig.savefig(image_path, bbox_inches='tight', dpi=300)
#         fig.show()
#         plt.close(fig)
#         print(f"Visualization saved to: {image_path}")
#         return image_path

#     except Exception as e:
#         print("Error during visualization:")
#         return None

def visualize_data(code: str):
    """
    Executes code for seaborn/matplotlib plot using global df,
    saves the figure to disk, and returns image path.
    """
    global df
    image_filename = "seaborn_plot.png"
    image_path = os.path.join("plots", image_filename)
    
    try:
        print("Generated Code:\n", code)
        
        # Clear any existing plots
        plt.clf()
        
        # Execute the plotting code
        exec(code)
        
        # Get current figure
        fig = plt.gcf()
        
        # Ensure we have a valid plot
        if not fig.get_axes():
            print("Warning: No plot was created")
            return None
        
        # Create directory
        os.makedirs("plots", exist_ok=True)
        
        # Save with explicit parameters
        plt.savefig(image_path, 
                   bbox_inches='tight', 
                   dpi=300, 
                   facecolor='white',
                   edgecolor='none',
                   format='png')
        
        plt.show()
        plt.close()
        
        print(f"Visualization saved to: {image_path}")
        return image_path
        
    except Exception as e:
        print(f"Error during visualization: {e}")
        plt.close('all')
        return None

In [8]:
tools = {}
tools["query_dataframe"] = FunctionTool.from_defaults(
        fn=query_dataframe,
        name="query_dataframe",
        description="Use this tool to query the pandas dataframe and return the filtered dataframe we got after the query"
        )

tools["visualize_data"] = FunctionTool.from_defaults(
        fn=visualize_data,
        name="visualize_data",
        description="takes the seaborn code for visualization and return the image path"
)

In [9]:
tool_map = {tool: tools[tool] for tool in tools.keys()}
logging.info(f"Tool map keys: {tool_map.keys()}")
tool_map

{'query_dataframe': <llama_index.core.tools.function_tool.FunctionTool at 0x243268650d0>,
 'visualize_data': <llama_index.core.tools.function_tool.FunctionTool at 0x24302096450>}

In [10]:
func = tool_map["query_dataframe"]

In [11]:
df = pd.read_excel("Data/TransactionReport14.07.2025 .xls")
#normalizing columns
df.columns = [col.strip(" ").replace(" ", "_").lower() for col in df.columns]

d:\SandBox\DA_Agent\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:


result = func("df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'district', 'taluka', 'village']].head(5)")

Generated Code:  df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'district', 'taluka', 'village']].head(5)


In [13]:
result.raw_output

,name_of_dam,district,taluka,village
0,Aadale M.I.,Pune,Mawal,Adhale bk
1,Aad M. I. Tank,Nashik,Peint,Aad Bk.
3,Aaland M. I.,Chhatrapati Sambhajinagar,Phulambri,Alad
4,Abhalwadi KT Weir,Ahilyanagar,Sangamner,Abhalwadi
5,Abhanpur M. I. Tank,Dhule,Shirpur,Abhanpur Kh.


In [14]:
vis_func = tool_map["visualize_data"]

In [15]:
visualization_code = """import matplotlib.pyplot as plt
import seaborn as sns

df_sorted = df.sort_values(by='live_storage_(tmc)', ascending=False).head(5)
plt.figure(figsize=(10, 6))
sns.barplot(x='name_of_dam', y='live_storage_(tmc)', data=df_sorted)
plt.title('Top 5 Dams by Live Storage (TMC)')
plt.xlabel('Name of Dam')
plt.ylabel('Live Storage (TMC)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()"""

vis_func(visualization_code)

Generated Code:
 import matplotlib.pyplot as plt
import seaborn as sns

df_sorted = df.sort_values(by='live_storage_(tmc)', ascending=False).head(5)
plt.figure(figsize=(10, 6))
sns.barplot(x='name_of_dam', y='live_storage_(tmc)', data=df_sorted)
plt.title('Top 5 Dams by Live Storage (TMC)')
plt.xlabel('Name of Dam')
plt.ylabel('Live Storage (TMC)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


<string>:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


Visualization saved to: plots\seaborn_plot.png


C:\Users\athar\AppData\Local\Temp\ipykernel_13360\3539981648.py:62: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


ToolOutput(blocks=[TextBlock(block_type='text', text='plots\\seaborn_plot.png')], tool_name='visualize_data', raw_input={'args': ("import matplotlib.pyplot as plt\nimport seaborn as sns\n\ndf_sorted = df.sort_values(by='live_storage_(tmc)', ascending=False).head(5)\nplt.figure(figsize=(10, 6))\nsns.barplot(x='name_of_dam', y='live_storage_(tmc)', data=df_sorted)\nplt.title('Top 5 Dams by Live Storage (TMC)')\nplt.xlabel('Name of Dam')\nplt.ylabel('Live Storage (TMC)')\nplt.xticks(rotation=45, ha='right')\nplt.tight_layout()\nplt.show()",), 'kwargs': {}}, raw_output='plots\\seaborn_plot.png', is_error=False)

In [16]:
with open(os.path.join(os.getcwd(), "config.json"), 'r') as f:
    config = json.loads(f.read())

llm = GoogleGenAI(
    model=config["llm"]["model"],
    temperature=config["llm"]["temperature"],
    api_key=os.getenv("GOOGLE_API_KEY"),
)

In [17]:
system_prompt = f"""
            You are a Data Analysis Agent which Analyzes the Pandas Dataframe and provide with the insights and Visualization tools. and tells your findings honestly

            You have tool like:
            query_dataframe -> which allows you to query the pandas dataframe using code and provide with the insights
            visualize_data -> which provide you with the seaborn visualization

            Columns of the dataframe are : {df.columns}

            HARD RULES:
            - Use only the column provided not invent other column name.
        """

In [18]:
prompt = "Show me the top 5 rows where lake_level_reading(rl) are greater than 90. just provide its name and location columns like taluka, district and region"

In [19]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content=system_prompt),
    ChatMessage(role="user", content=prompt),
]

In [20]:
response = llm.chat_with_tools(
                chat_history=messages,
                tools=list(tools.values()),
                allow_parallel_tool_calls=False,
                verbose=True,
    )

In [21]:
tool_calls = response.message.additional_kwargs.get("tool_calls", [])
logging.info(f"Tool calls: {tool_calls}")
if not tool_calls:
    logging.info("No tool calls in LLM response")

In [22]:
tool_calls = response.message.additional_kwargs.get("tool_calls", [])
print(f"Tool calls: {tool_calls}")
if not tool_calls:
    print("No tool calls in LLM response")

Tool calls: [FunctionCall(
  args={
    'code': "df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'taluka', 'district', 'region']].head(5)"
  },
  name='query_dataframe'
)]


In [23]:
for tool_call in tool_calls:
    tool_name = tool_call.name
    print(f"Processing tool call: {tool_name}")
    tool_func = tool_map[tool_name]
    print(tool_func)
    if not tool_func:
        print(f"Tool {tool_name} not found in tool_map: {tool_map.keys()}")
        continue
    try:
        if tool_name == "query_dataframe":
            args = tool_call.args
            code = args["code"]
            if not isinstance(code, str) or not code.strip():
                print(f"Skipping {tool_name} with invalid or empty code: {code}")
                continue
            result = tool_func(code)
            
            print(f"Executing tool {tool_name} with Code: {code}")
            print(f"Tool {tool_name} result: {result}")
    except Exception as e:
        print(f"Error executing tool {tool_name}: {type(e).__name__}: {e}")

Processing tool call: query_dataframe
Generated Code:  df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'taluka', 'district', 'region']].head(5)
Executing tool query_dataframe with Code: df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'taluka', 'district', 'region']].head(5)
Tool query_dataframe result:            name_of_dam     taluka                   district  \
0          Aadale M.I.      Mawal                       Pune   
1       Aad M. I. Tank      Peint                     Nashik   
3         Aaland M. I.  Phulambri  Chhatrapati Sambhajinagar   
4    Abhalwadi KT Weir  Sangamner                Ahilyanagar   
5  Abhanpur M. I. Tank    Shirpur                      Dhule   

                                              region  
0  Chief Engineer, Water Resources Department - P...  
1  Chief Engineer, Water Resources Department, Ko...  
3  Chief Engineer and Chief Administrator, Water ...  
4  Chief Engineer, North Maharashtra Region - Nas...  
5  Chief Engineer, Tapi 

In [24]:
code = tool_calls[0].args.get("code", "")
code

"df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'taluka', 'district', 'region']].head(5)"

In [25]:
tool_map["query_dataframe"](code)

Generated Code:  df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'taluka', 'district', 'region']].head(5)


ToolOutput(blocks=[TextBlock(block_type='text', text='           name_of_dam     taluka                   district  \\\n0          Aadale M.I.      Mawal                       Pune   \n1       Aad M. I. Tank      Peint                     Nashik   \n3         Aaland M. I.  Phulambri  Chhatrapati Sambhajinagar   \n4    Abhalwadi KT Weir  Sangamner                Ahilyanagar   \n5  Abhanpur M. I. Tank    Shirpur                      Dhule   \n\n                                              region  \n0  Chief Engineer, Water Resources Department - P...  \n1  Chief Engineer, Water Resources Department, Ko...  \n3  Chief Engineer and Chief Administrator, Water ...  \n4  Chief Engineer, North Maharashtra Region - Nas...  \n5  Chief Engineer, Tapi Irrigation Development Co...  ')], tool_name='query_dataframe', raw_input={'args': ("df[df['lake_level_reading(rl)'] > 90][['name_of_dam', 'taluka', 'district', 'region']].head(5)",), 'kwargs': {}}, raw_output=           name_of_dam     taluka      

In [ ]:
type(result.raw_output)

pandas.core.frame.DataFrame

: 